# RNN from scratch

Notebook normalement peu utile si vous avez eu le cours... Sinon, c'est une manière d'aborder d'intercaler une marche avant le premier notebook sur les séries temporelles

In [1]:
from utils import RNN, device,SampleMetroDataset
import torch
from torch.utils.data import DataLoader

In [2]:
import numpy as np
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
import logging
import tensorboard
from torch.utils.tensorboard import SummaryWriter
import time
from itertools import chain
logging.basicConfig(level=logging.INFO)


## Première idée: construire la cellule récurrente

C'est ce qui est fait dans les notebooks des tuto officiels (remarquables) de pytorch:
-   [NLP From Scratch: Classifying Names with a Character-Level
    RNN](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)
-   [NLP From Scratch: Generating Names with a Character-Level
    RNN](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html)

On rappelle la structure classique en classification de signaux (many-to-one):

![Image RNN](data/rnn_unfold2.png)

## Création du réseau

1. Réseau correspondant au besoin ci-dessus
2. Données arbitraires
3. Instanciation du réseau + utilisation sur un exemple
4. Passage dans une loss pour vérifier la cohérence des dimensions

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class RNN_V1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_V1, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size) # W1
        self.h2h = nn.Linear(hidden_size, hidden_size) # W2
        self.h2o = nn.Linear(hidden_size, output_size) # W3
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden=None):
        if hidden==None: hidden = self.initHidden() # init par défaut à 0
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


In [4]:
# 2. Data

# data (x_1,...,x_T), batch, dimension des entrées
#   dim1 : Seq. length T=23 
#   dim2 : batch of size 10
#   dim3 : Input x_t \in R^n dimension 100
seq_x = torch.rand(23, 10, 100) # simulation d'un batch
y     = torch.randint(0,2,(10,))

print(y)


tensor([0, 1, 1, 1, 0, 0, 1, 1, 0, 1])


In [5]:

n_hidden = 50
d_data   = 100
n_cl     = 2
rnn = RNN_V1(d_data, n_hidden, n_cl)

## Faire passer les données dans le réseau...

Qu'est ce que j'ai le droit de donner à ce réseau??

Avez-vous remarqué qu'il n'est pas récurrent??

Sélectionner une propostion parmi les deux boites suivantes en réfléchissant aux conséquences.

In [6]:
# PROPOSITION 1:
yhat, hidden = rnn(seq_x)

print("yhat: ",yhat.size())
print("hidden: ",hidden.size())

yhat:  torch.Size([23, 10, 2])
hidden:  torch.Size([23, 10, 50])


In [7]:
# PROPOSITION 2:
yhat, hidden = rnn(seq_x[0,:,:])

print("yhat: ",yhat.size())
print("hidden: ",hidden.size())

yhat:  torch.Size([10, 2])
hidden:  torch.Size([10, 50])


## RNN et récurrence

Evidemment, traiter une séquence entière sans prendre en compte les états cachés n'a aucun intérêt... Mais si on ne traite que le premier état observé, on n'a pas fait le travail complètement non plus...

Complétons donc la solution précédente:

In [8]:
# Passage de l'ensemble de la séquence

hidden = rnn.initHidden() # en dehors de la boucle
for i in range(len(seq_x)):  # ca devrait marcher, le temps est stocké dans la première dim
    yhat, hidden = rnn(seq_x[i,:,:], hidden) # iteration sur les états cachés calculés

print("yhat: ",yhat.size())
print("hidden: ",hidden.size())

yhat:  torch.Size([10, 2])
hidden:  torch.Size([10, 50])


## RNN V2

La solution actuelle est insatisfaisante: le `forward` est non standard, on ne peut pas intégrer cette architecture dans notre boucle classique de train...
On a donc besoin d'une nouvelle version.

Intégrer la boucle de récurrence à l'interieur du forward pour avoir:
- une séquence en entrée
- une séquence en sortie (toutes les couches cachées) + une sortie

In [9]:
class RNN_V2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_V2, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size) # W1
        self.h2h = nn.Linear(hidden_size, hidden_size) # W2
        self.h2o = nn.Linear(hidden_size, output_size) # W3
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden=None):
        if hidden==None: hidden = self.initHidden() # init par défaut à 0
        allh = []

        # calculer tous les états cachés
        # les ajouter dans la liste
        # Former un tensor avec tous les éléments de la liste (torch.cat)
        ## <CORRECTION>
        for i in range(len(input)):  # ca devrait marcher, le temps est stocké dans la première dim
            hidden = F.tanh(self.i2h(input[i,:,:]) + self.h2h(hidden))
            allh.append(hidden.unsqueeze(0))
        allh = torch.cat(allh, dim=0)
        ## </CORRECTION>
      
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, allh

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


Vérifier les dimensions attendues dans la boite ci-dessous

In [10]:
rnn = RNN_V2(d_data, n_hidden, n_cl)

yhat, hidden = rnn(seq_x)

print("yhat: ",yhat.size())
print("hidden: ",hidden.size())

yhat:  torch.Size([10, 2])
hidden:  torch.Size([23, 10, 50])


## RNN V3

Utiliser un module prêt à l'emploi RNN, (on basculera ensuite vers des LSTM ou des GRU facilement)

Après le rappel de cours, utiliser ce module à l'intérieur de votre proposition.

[Lien vers la doc](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)


In [ ]:
# Rappel de cours

model = torch.nn.RNN(input_size=100, hidden_size=50, 
                     num_layers=1,
                     nonlinearity='tanh', bias=True)

# data (x_1,...,x_T), batch, dimension des entrées
#   dim1 : Seq. length T=23 
#   dim2 : batch of size 10
#   dim3 : Input x_t \in R^n dimension 100
seq_x = torch.rand(23, 10, 100) # simulation d'un batch

# For classification purpose, we use last_h
seq_h, last_h = model(seq_x)

print("seq_h : ",seq_h.size())
print("last_h: ", last_h.size())

seq_h :  torch.Size([23, 10, 50])
last_h:  torch.Size([1, 10, 50])


In [ ]:
class RNN_V3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_V3, self).__init__()

        self.hidden_size = hidden_size
        self.rec = nn.RNN(input_size=input_size, hidden_size=hidden_size, 
                     num_layers=1,
                     nonlinearity='tanh') # activation par défaut... On peut en tester d'autres
        self.h2o = nn.Linear(hidden_size, output_size) # W3
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden=None):
        
        # calculer tous les états cachés d'un coup
        # le RNN de torch accepte None comme valeur par défaut pour la couche cachée 0...
        # 1 seule ligne est attendue dans la correction
        ## <CORRECTION>
        allh, hidden = self.rec(input, hidden)
        ## </CORRECTION>
      
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, allh

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


Vérifier les dimensions attendues dans la boite ci-dessous

In [16]:
rnn = RNN_V3(d_data, n_hidden, n_cl)

yhat, hidden = rnn(seq_x)

print("yhat: ",yhat.size())
print("hidden: ",hidden.size())

yhat:  torch.Size([1, 10, 2])
hidden:  torch.Size([23, 10, 50])


## Tests en suspend

- Tester le passage au GRU ou LSTM: attention, il y a une légère variation dans les arguments de sortie du `forward` de ces modèles.
- Cette architecture n'est pas compatible avec les applications many-to-many ni avec la `CrossEntropyLoss`: envisager des solutions

# Construction du sujet à partir de la correction

In [17]:
### <CORRECTION> ###
import re
# transformation de cet énoncé en version étudiante

fname = "4_0_RNN-archi-corr.ipynb" # ce fichier
fout  = fname.replace("-corr","")

# print("Fichier de sortie: ", fout )

f = open(fname, "r")
txt = f.read()
 
f.close()

f2 = open(fout, "w")
f2.write(re.sub("<CORRECTION>.*?(</CORRECTION>)"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###